In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
			)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__

live = client.is_live()
connected = client.is_connected()

print(f"Weaviate client is ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"live: {live}")
print(f"Connected: {connected}")

In [ ]:
# Existing of a collection
col = client.collections.get("<COLLECTION_NAME>")
if col.exists():
    print("Collection exists")
else:
    print("Collection does not exist")

In [ ]:
# Query an object & its dimensionality
coll = client.collections.get("<COLLECTION_NAME>")

some_data = coll.query.fetch_objects(include_vector=True, limit=1)
default_vector = some_data.objects[0].vector["default"]
dimensionality = len(default_vector)

print(f"The default vector has {dimensionality} dimensions.")

In [ ]:
# Query an object & its dimensionality for a Tenant
coll = client.collections.get("<COLLECTION_NAME>")
tenants = coll.tenants.get()

# Extract the first tenant's name
first_tenant = list(tenants.values())[0]  # Get the first Tenant object
tenant_name = first_tenant.name  # Extract the tenant name

# Get the collection specific to this tenant
tenant_coll = coll.with_tenant(tenant_name)

# Fetch an object for the specific tenant
tenant_object = tenant_coll.query.fetch_objects(include_vector=True, limit=1)

# Extract and compute the vector dimensionality
default_vector = tenant_object.objects[0].vector["default"]
dimensionality = len(default_vector)

print(f"The default vector has {dimensionality} dimensions for tenant '{tenant_name}'.")

In [ ]:
# List the collection names and their dimensions in Weaviate cluster
try:
    collections = client.collections.list_all()
    if collections:
        print("Collections and their dimensionality in Weaviate:")
        
        # Loop through each collection in the instance
        for collection_name in collections.keys():         
            coll = client.collections.get(collection_name)           
            try:
                # Try checking if the collection has tenants (Multi-Tenancy enabled)
                try:
                    tenants = coll.tenants.get()
                    is_multi_tenant = True
                except Exception:
                    is_multi_tenant = False 
                if is_multi_tenant:
                    # Multi-Tenant Collection: Get first available tenant
                    if tenants:
                        first_tenant = list(tenants.values())[0]  # Get first tenant object
                        tenant_name = first_tenant.name  # Extract tenant name
                        # Get the collection specific to this tenant
                        tenant_coll = coll.with_tenant(tenant_name)
                        # Fetch an object for the tenant
                        some_data = tenant_coll.query.fetch_objects(include_vector=True, limit=1)
                    else:
                        print(f"Collection: {collection_name} - No tenants found.")
                        continue
                else:
                    # Standard Collection (Non-MT)
                    some_data = coll.query.fetch_objects(include_vector=True, limit=1) 
                # Process fetched object
                if some_data.objects:
                    default_vector = some_data.objects[0].vector["default"]
                    dimensionality = len(default_vector)
                    print(f"Collection: {collection_name} - Dimensionality: {dimensionality}")
                else:
                    print(f"Collection: {collection_name} - No objects found.")
            except Exception as inner_e:
                print(f"Collection: {collection_name} - Error fetching objects: {inner_e}")
    else:
        print("No collections found.")
except Exception as e:
    print(f"Error retrieving collections or dimensions: {e}")

In [ ]:
# Query & Filter by time
from datetime import datetime
from weaviate.classes.query import Filter

collection = client.collections.get("<COLLECTION_NAME>")

filter_time = datetime(2024, 1, 1).isoformat()
print(filter_time)
res = collection.query.fetch_objects(
    filters=Filter.by_creation_time().greater_than(filter_time)
)
print(res)

In [ ]:
# Sorting -> works only with fetch_objects (Similarity searches do not support sorting as they are based sorted by similarity)
from weaviate.classes.query import Sort

movies_col= client.collections.get("<COLLECTION_NAME>")

response = movies_col.query.fetch_objects(
    limit=9,
    sort=Sort.by_property(name="<PROP_NAME>", ascending=False)
)

for o in response.objects:
    print(o.properties)